### Molecular docking

In [4]:
import pandas as pd
import vina

#### Prepare ligands

In [5]:
ligands = ['ddt', 'permethrin', 'deltamethrin']

In [5]:
for ligand in ligands:
    !mk_prepare_ligand.py -i "../data/ligands/{ligand}.sdf" -o "../data/ligands/{ligand}.pdbqt"

Traceback (most recent call last):
  File "/Users/dennistpw/miniconda3/envs/proteindocking/bin/mk_prepare_ligand.py", line 286, in <module>
    mol_supplier = parsers[ext](input_molecule_filename, removeHs=False) # input must have explicit H
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: File error: Bad input file ../data/ligands/ligand_deltamethrin.sdf


### Prepare receptors

In [ ]:
#receptor list = ['agap006227']

In [32]:
! ~/apps/ADFRsuite-1.0/bin/prepare_receptor -r ../../../results/protein_modelling/prediction/selected_prediction.pdb -o ../../../results/protein_modelling/agap006227.pdbqt

adding gasteiger charges to peptide


In [11]:
for idx, lig in ligands.iterrows():

    liga = lig['filename']
    prefix = lig['ligand']
    #! ls "/home/sanj/projects/coeae1f/results/protein_modelling/"
    ! pythonsh prepare_gpf.py -l ligands/{prefix}.pdbqt -r agap006227.pdbqt -y -o agap006227_{prefix}.gpf
    ! autogrid4 -p agap006227_{prefix}.gpf -l agap006227_{prefix}.glg

### Run Vina

In [2]:
from rdkit import Chem
from rdkit.Chem.rdMolTransforms import ComputeCentroid
from vina import Vina
v = Vina(sf_name='vina')

In [6]:
receptor = 'agap006227'
v.set_receptor(f'{receptor}.pdbqt')

for idx, lig in ligands.iterrows():
    sdf = lig['filename']
    ligand = lig['ligand']

    lig = Chem.SDMolSupplier(f'/home/sanj/projects/coeae1f/results/protein_modelling/ligands/{sdf}', removeHs=False)[0]
    centroid = ComputeCentroid(lig.GetConformer())
    print(centroid.x, centroid.y, centroid.z)

    with open(f"{receptor}_{ligand}.config", 'w') as outfile:
        outfile.write(f"center_x = {centroid.x}\ncenter_y = {centroid.y}\ncenter_z = {centroid.z}\nsize_x = 20.0\nsize_y = 20.0\nsize_z = 20.0")
    
    !vina --receptor {receptor}.pdbqt --ligand ligands/{ligand}.pdbqt  --config {receptor}_{ligand}.config --exhaustiveness=8 --out {receptor}_{ligand}.out.pdbqt > {receptor}_{ligand}.log

0.0 -6.2500000000131894e-06 -1.0408340855860843e-17
4.18415625 -0.625 0.0
8.69565217395553e-06 -4.347826086985005e-06 -4.347826086946389e-06
5.240921739130434 0.3903478260869565 0.0
3.846153846281525e-06 -7.692307692375167e-06 4.270088556250602e-18
5.236453846153847 -0.0642307692307693 0.0
-6.6666666666511295e-06 -6.666666666665007e-06 6.666666666673334e-06
5.105966666666666 0.26342666666666664 0.0
4.233421052631579 0.40052631578947356 0.0
3.571428571420248e-06 -3.172065784643304e-17 -7.142857142822653e-06
5.170435714285716 0.6776428571428573 0.0
-5.882352941215007e-06 1.1764705882377768e-05 -1.1764705882338585e-05
3.949464705882353 0.05199999999999999 0.0
-5.263157894712889e-06 -1.0526315789472526e-05 -5.2631578947012025e-06
5.600836842105264 0.6725210526315791 0.0
-5.263157894771322e-06 5.263157894759636e-06 1.0526315789460838e-05
5.919894736842107 -0.2885263157894737 0.0
-1.2500000000044883e-05 4.166666666670833e-06 8.333333333313911e-06
4.453729166666667 0.16666666666666666 0.0
3.2

In [ ]:
['AGAP004707-RD'] #vgsc gam wt


In [ ]:
receptor = 'agap006227'
v.set_receptor(f'{receptor}.pdbqt')

receptor_center = [15.190, 53.903, 16.917]
box_size = [20, 20, 20]

for idx, lig in ligands.iterrows():

    liga = lig['filename']
    prefix = lig['ligand']

    v.set_ligand_from_file(f'ligands/{prefix}.pdbqt')
    v.compute_vina_maps(center=receptor_center, box_size=box_size)
    # Score the current pose
    energy = v.score()
    print('Score before minimization: %.3f (kcal/mol)' % energy[0])

    # Minimized locally the current pose
    energy_minimized = v.optimize()
    print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
    v.write_pose(f'{receptor}_{prefix}.minimised.pdbqt', overwrite=True)

    # Dock the ligand
    v.dock(exhaustiveness=32, n_poses=20)
    v.write_poses(f'{receptor}_{prefix}.fullout.pdbqt', n_poses=10, overwrite=True)
